In [253]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models

from collections import OrderedDict

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

def get_label(activity_type, activity_subtype):
    return f'{activity_type} - {activity_subtype}'

file_dir = '/Users/julius/uni/pdiot/cw3/2023/all/'

columns = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

labels = []
feature_accel_x = []
feature_accel_y = []
feature_accel_z = []
feature_gyro_x = []
feature_gyro_y = []
feature_gyro_z = []
test_labels = []
test_feature_accel_x = []
test_feature_accel_y = []
test_feature_accel_z = []
test_feature_gyro_x = []
test_feature_gyro_y = []
test_feature_gyro_z = []

for filename in os.listdir(file_dir):
    match = re.fullmatch('^Respeck_s\d{7}_(.*)_(.*)_clean_\d{2}-\d{2}-\d{4}_\d{2}-\d{2}-\d{2}.csv$', filename)
    
    if match:
        label = get_label(match.group(1), match.group(2))
        
        dataframe = pd.read_csv(os.path.join(file_dir, filename))
        data = get_data(dataframe)
        
        for i in range(6):
            labels.append(label)
            feature_accel_x.append(dataframe[i * 100:(i + 1) * 100]['accel_x'].values.astype('float32'))
            feature_accel_y.append(dataframe[i * 100:(i + 1) * 100]['accel_y'].values.astype('float32'))
            feature_accel_z.append(dataframe[i * 100:(i + 1) * 100]['accel_z'].values.astype('float32'))
            feature_gyro_x.append(dataframe[i * 100:(i + 1) * 100]['gyro_x'].values.astype('float32'))
            feature_gyro_y.append(dataframe[i * 100:(i + 1) * 100]['gyro_y'].values.astype('float32'))
            feature_gyro_z.append(dataframe[i * 100:(i + 1) * 100]['gyro_z'].values.astype('float32'))
            
        test_feature_accel_x.append(dataframe[600:700]['accel_x'].values.astype('float32'))
        test_feature_accel_y.append(dataframe[600:700]['accel_y'].values.astype('float32'))
        test_feature_accel_z.append(dataframe[600:700]['accel_z'].values.astype('float32'))
        test_feature_gyro_x.append(dataframe[600:700]['gyro_x'].values.astype('float32'))
        test_feature_gyro_y.append(dataframe[600:700]['gyro_y'].values.astype('float32'))
        test_feature_gyro_z.append(dataframe[600:700]['gyro_z'].values.astype('float32'))
        test_labels.append(label)

int_label_dict = dict(enumerate(set(labels)))
label_int_dict = dict([(label, index) for index, label in enumerate(set(labels))])

test_labels = np.array([label_int_dict[label] for label in test_labels])

labels = np.array([label_int_dict[label] for label in labels])

In [254]:
feature_accel_x = np.array(feature_accel_x)
feature_accel_y = np.array(feature_accel_y)
feature_accel_z = np.array(feature_accel_z)
feature_gyro_x = np.array(feature_gyro_x)
feature_gyro_y = np.array(feature_gyro_y)
feature_gyro_z = np.array(feature_gyro_z)
test_feature_accel_x = np.array(test_feature_accel_x)
test_feature_accel_y = np.array(test_feature_accel_y)
test_feature_accel_z = np.array(test_feature_accel_z)
test_feature_gyro_x = np.array(test_feature_gyro_x)
test_feature_gyro_y = np.array(test_feature_gyro_y)
test_feature_gyro_z = np.array(test_feature_gyro_z)

In [255]:
input1 = layers.Input(shape=(100,))
input2 = layers.Input(shape=(100,))
input3 = layers.Input(shape=(100,))
input4 = layers.Input(shape=(100,))
input5 = layers.Input(shape=(100,))
input6 = layers.Input(shape=(100,))
merged = layers.Concatenate(axis=1)([input1, input2, input3, input4, input5, input6])
dense1 = layers.Dense(96, input_dim=6, activation='sigmoid', use_bias=True)(merged)
output = layers.Dense(44, activation='relu', use_bias=True)(dense1)
m = models.Model(inputs=[input1, input2, input3, input4, input5, input6], outputs=output)

In [256]:
m.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_152 (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 input_153 (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 input_154 (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 input_155 (InputLayer)         [(None, 100)]        0           []                               
                                                                                           

In [257]:
m.compile(
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['mae'],
    loss='mse',
    #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [258]:
m.fit(x=[feature_accel_x, feature_accel_y, feature_accel_z, feature_gyro_x, feature_gyro_y, feature_gyro_z], y=labels, batch_size=1, validation_split=0.2, epochs=10)


Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
614/633 [============================>.] - ETA: 0s - loss: 307.4849 - mae: 14.0235WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f7bceeffe60> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
633/633 [==============================] - 3s 3ms/step - loss: 305.2054 - mae: 13.9424 - val_loss: 202.7047 - val_mae: 11.7523
Epoch 2/10
633/633 [==============================] - 2s 3ms/step - loss: 155.0770 - mae: 10.1804 - val_loss: 199.3684 - val_mae: 11.5855
Epoch 3/10
633/633 [==============================] - 2s 3ms/step - loss: 136.3420 - mae: 9.4895 - val_loss: 199.3642 - val_mae: 11.5257
Epoch 4/10
633/633 [==============================] - 2s 3ms/step - loss: 123.0550 - mae: 8.9829 - val_loss: 187.4279 - val_mae: 11.1611
Epoch 5/10
633/633 [==============================] - 2s 3ms/step - loss: 115.2766 - mae: 8.6577 - val_loss: 200.0402 - val_mae: 11.6462
Epoch 6/10
63

In [259]:
m.evaluate(x=[test_feature_accel_x, test_feature_accel_y, test_feature_accel_z, test_feature_gyro_x, test_feature_gyro_y, test_feature_gyro_z], y=test_labels, return_dict=True)


5/5 [==============================] - 0s 3ms/step - loss: 172.8929 - mae: 10.7972


{'loss': 172.8929443359375, 'mae': 10.797218322753906}

In [275]:
filename = "/Users/julius/uni/pdiot/cw3/2023/s2604021/Respeck_s2604021_Descending stairs_Normal_clean_02-10-2023_14-57-44.csv"

df = pd.read_csv(filename)

accel_x = dataframe[400:500]['accel_x'].values.astype('float32')
accel_y = dataframe[400:500]['accel_y'].values.astype('float32')
accel_z = dataframe[400:500]['accel_z'].values.astype('float32')
gyro_x = dataframe[400:500]['gyro_x'].values.astype('float32')
gyro_y = dataframe[400:500]['gyro_y'].values.astype('float32')
gyro_z = dataframe[400:500]['gyro_z'].values.astype('float32')




In [276]:
predict = [
    np.expand_dims(accel_x, axis=0),
    np.expand_dims(accel_y, axis=0),
    np.expand_dims(accel_z, axis=0),
    np.expand_dims(gyro_x, axis=0),
    np.expand_dims(gyro_y, axis=0),
    np.expand_dims(gyro_z, axis=0)
]

In [277]:
p = m.predict(predict)

1/1 [==============================] - 0s 25ms/step


In [278]:
p

array([[18.618546, 18.743603, 18.679111, 18.71624 , 18.13827 , 18.630268,
        18.984854, 18.651892, 18.784233, 18.479322, 19.279171, 19.715895,
        18.941048, 18.918516, 19.08693 , 18.875355, 18.8701  , 18.811092,
        18.792288, 18.594885, 19.136677, 18.706364, 18.778086, 19.130554,
        18.853632, 19.087702, 18.745512, 18.681536, 18.894423, 18.686249,
        18.407085, 18.38813 , 18.452389, 18.989107, 18.58169 ,  0.      ,
        18.632214, 18.757702, 18.556604, 18.485228, 18.335081, 18.790718,
        18.199078, 18.739573]], dtype=float32)

In [279]:
int_label_dict[np.argmax(p[0])]


'Normal walking - Normal'

In [286]:
converter = tf.lite.TFLiteConverter.from_keras_model(m)
tflite_model = converter.convert()

model_file = '/Users/julius/uni/pdiot/cw3/model6features.tflite'
# Save the model.
with open(model_file, 'wb') as f:
    f.write(tflite_model)
    

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: /var/folders/b6/ldk_ls6j78l1yw3kdcf9_tgw0000gn/T/tmp6wvg79sb/assets


INFO:tensorflow:Assets written to: /var/folders/b6/ldk_ls6j78l1yw3kdcf9_tgw0000gn/T/tmp6wvg79sb/assets
2023-10-17 23:41:31.642508: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-10-17 23:41:31.643419: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-10-17 23:41:31.646746: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/b6/ldk_ls6j78l1yw3kdcf9_tgw0000gn/T/tmp6wvg79sb
2023-10-17 23:41:31.651137: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-10-17 23:41:31.651184: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/b6/ldk_ls6j78l1yw3kdcf9_tgw0000gn/T/tmp6wvg79sb
2023-10-17 23:41:31.664041: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-10-17 23:41:31.784085: I tensorflow/cc/saved_model/loader.cc:213] Running initialization

In [285]:
label_file = '/Users/julius/uni/pdiot/cw3/labels.txt'
with open(label_file, 'w') as f:
    f.write(str(int_label_dict))